In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import acnportal
import sklearn
from matplotlib import pyplot as plt
import matplotlib
import matplotlib.dates as mdates
import seaborn as sns

from copy import deepcopy
import warnings
import pytz
import time
import numpy as np
import pandas as pd
import pickle
from datetime import datetime, timedelta
from enum import Enum
from collections import namedtuple, defaultdict
import gzip
import random
import os
import json

from acnportal import acnsim
from acnportal import algorithms
from acnportal.acnsim.events import EventQueue
# from acnportal.signals.tariffs.tou_tariff import TimeOfUseTariff
from utility_functions.signals.tariffs.tou_tariff import TimeOfUseTariff
from acnportal.acnsim.events import GaussianMixtureEvents
from acnportal.contrib.acnsim import StochasticNetwork
from acnportal.acnsim.network import ChargingNetwork
from modified_adacharge import modified_adacharge
from modified_simulator import modified_simulator
# from acnportal.acnsim import analysis
from utility_functions import analysis
# from modified_evse import *
from acnportal.acnsim.models.evse import get_evse_by_type
# import sqlite3
# from sqlalchemy import create_engine

In [3]:
# This method won't take a evse_per_phase argument
def ev_fleet_level_2_network(transformer_cap=130):
    """ Configurable charging network for level-2 EVSEs connected line to line
        at 415 V.

    Args:
        transformer_cap (float): Capacity of the transformer feeding the network
          [kW]
        evse_per_phase (int): Number of EVSEs on each phase. Total number of
          EVSEs will be 3 * evse_per_phase.

    Returns:
        ChargingNetwork: Configured ChargingNetwork.
    """
    network = StochasticNetwork(early_departure=True)
    # network = ChargingNetwork()
    voltage = 415
    evse_type = 'AeroVironment'
    # evse_type = 'ClipperCreek'

    # Define the sets of EVSEs in the Caltech ACN.
    AB_ids = ['AB-{0}'.format(i) for i in range(5)]
    BC_ids = ['BC-{0}'.format(i) for i in range(5)]
    CA_ids = ['CA-{0}'.format(i) for i in range(5)]

    # print(AB_ids)

    # Add Caltech EVSEs
    for evse_id in AB_ids:
        network.register_evse(get_evse_by_type(evse_id, evse_type), voltage, 30)
    for evse_id in BC_ids:
        network.register_evse(get_evse_by_type(evse_id, evse_type), voltage, -90)
    for evse_id in CA_ids:
        network.register_evse(get_evse_by_type(evse_id, evse_type), voltage, 150)

    # Add Caltech Constraint Set
    AB = acnsim.Current(AB_ids)
    BC = acnsim.Current(BC_ids)
    CA = acnsim.Current(CA_ids)

    # Define intermediate currents
    I3a = AB - CA
    I3b = BC - AB
    I3c = CA - BC
    I2a = (1 / 4) * (I3a - I3c)
    I2b = (1 / 4) * (I3b - I3a)
    I2c = (1 / 4) * (I3c - I3b)

    # Build constraint set
    primary_side_constr = transformer_cap * 1000 / 3 / 415
    print(primary_side_constr)
    secondary_side_constr = transformer_cap * 1000 / 3 / 230
    print(secondary_side_constr)
    network.add_constraint(I3a, secondary_side_constr, name='Secondary A')
    network.add_constraint(I3b, secondary_side_constr, name='Secondary B')
    network.add_constraint(I3c, secondary_side_constr, name='Secondary C')
    network.add_constraint(I2a, primary_side_constr, name='Primary A')
    network.add_constraint(I2b, primary_side_constr, name='Primary B')
    network.add_constraint(I2c, primary_side_constr, name='Primary C')

    return network

In [4]:
# How long each time discrete time interval in the simulation should be.
PERIOD = 5  # minutes

# Voltage of the network.
VOLTAGE = 415  # volts      

# Default maximum charging rate for each EV battery.
DEFAULT_BATTERY_POWER = 11 # kW

#### Generate synthetic events for simulation

In [6]:
# Generate synthetic events based on JPL data provided by ACN-Sim.
class CustomUnpicklerJPLdata(pickle.Unpickler):
    def find_class(self, module, name):
        if name == "sklearn.mixture.gaussian_mixture":
            return sklearn.mixture.GaussianMixture
        if name == "GaussianMixture":
            return sklearn.mixture.GaussianMixture
        return super().find_class(module, name)
    
def get_synth_events(sessions_per_day):
    gmm = CustomUnpicklerJPLdata(open('./data/jpl_weekday_40.pkl', "rb")).load()


    # Generate a list of the number of sessions to draw for each day.
    # This generates 30 days of charging demands.
    # num_evs = [0]*2 + [sessions_per_day]*5 + [0]*2 + [sessions_per_day]*5 + [0]*2 + \
    #           [sessions_per_day]*5 + [0]*2 + [sessions_per_day]*5 + [0]*2

    # Generate sessions for 1 day (weekdays only)
    num_evs = [0]*2 + [sessions_per_day]*1

    # Note that because we are drawing from a distribution, some sessions will be
    # invalid, we ignore these sessions and remove the corresponding plugin events.
    # gen = GaussianMixtureEvents(pretrained_model=gmm, duration_min=0.08334)
    gen = GaussianMixtureEvents(pretrained_model=gmm, duration_min=5)

    synth_events = gen.generate_events(num_evs, PERIOD, VOLTAGE, DEFAULT_BATTERY_POWER)
    return synth_events

In [7]:
# 46 EV sessions per weekday
events_46_ev = get_synth_events(46)

c:\Users\s3955218\Anaconda3\envs\evsim\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator GaussianMixture from version 0.20.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [8]:
# Checking session details
session_copy = deepcopy(events_46_ev)
# print(type(session_copy))
session_list = list(session_copy.queue)
for session in session_list:
    print(session[0], session[1].ev._session_id, session[1].ev.arrival, \
          session[1].ev.departure, session[1].ev.requested_energy, session[1].event_type)

634 session_31 634 728 2.171414415102155 Plugin
642 session_38 642 702 13.96693671003839 Plugin
650 session_26 650 772 7.691942103577104 Plugin
646 session_12 646 770 11.821656636808518 Plugin
644 session_44 644 774 5.943028459109077 Plugin
652 session_4 652 772 9.039281566991093 Plugin
652 session_13 652 779 12.208305881364941 Plugin
654 session_10 654 752 8.586984437809045 Plugin
648 session_32 648 782 47.5322126616137 Plugin
660 session_39 660 776 13.526991440781607 Plugin
644 session_45 644 773 6.04067122678471 Plugin
669 session_24 669 770 6.43202169682417 Plugin
659 session_6 659 775 21.36574995981909 Plugin
659 session_27 659 781 26.92616131263511 Plugin
668 session_14 668 728 16.70568095983352 Plugin
660 session_15 660 720 9.266766560079565 Plugin
671 session_16 671 731 4.05140704949433 Plugin
668 session_35 668 776 10.154123423249654 Plugin
665 session_36 665 751 41.06431857119864 Plugin
663 session_40 663 766 15.57332621791023 Plugin
665 session_41 665 776 12.319527428241177 

#### Algorithms and new equation

In [9]:
sch = {}
sch['Unctrl'] = algorithms.UncontrolledCharging()
sch['RR'] = algorithms.RoundRobin(algorithms.first_come_first_served, continuous_inc=1)

cost_min_obj = [
                modified_adacharge.ObjectiveComponent(modified_adacharge.tou_energy_cost_with_pv),
                modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty),
                # modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty_without_priority_ev),
                # modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty_for_priority_ev),
                modified_adacharge.ObjectiveComponent(modified_adacharge.quick_charge, 1e-6)
               ]

sch['MPC'] = modified_adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj, solver="MOSEK", quantize=True, reallocate=False, peak_limit=1000, max_recompute=1)

#### Run Simulation with New Equation

In [34]:
sims = dict()
def run_experiment(alg_name, cap):
    """ Run simulation for the events defined previously and the specified
        network / algorithm / events.
    """
    # Timezone of the ACN we are using.
    # timezone = pytz.timezone('America/Los_Angeles')
    # Australian timezone for the experiment
    timezone = pytz.timezone('Australia/Melbourne')

    start = timezone.localize(datetime(2025, 7, 10))
    end = timezone.localize(datetime(2025, 7, 10))

    cn = ev_fleet_level_2_network(transformer_cap=cap)

    alg = deepcopy(sch[alg_name])
    alg.max_rate_estimator = algorithms.SimpleRampdown()
    alg.estimate_max_rate = True
    experiment_events = deepcopy(events_46_ev)
    signals = {'tariff': TimeOfUseTariff('sce_tou_ev_4_march_2019')}

    sim = modified_simulator.Simulator(cn, alg, experiment_events, start, period=PERIOD, signals=signals, verbose=False)
    print("Running...")
    start_simulation = time.time()
    if alg_name == "MPC_Offline":
        alg.register_events(experiment_events)
        alg.solve()
    # if alg_name == "MPC":
    #     sim.high_priority_ev_sessions = ["session_19", "session_7", "session_15", "session_10"]
    # sim.high_priority_ev_sessions = ["session_4", "session_12", "session_2"]
    sim.run()
    print(f"Run time: {time.time() - start_simulation}")

    return sim

In [11]:
warnings.simplefilter("ignore")

capacities = list(range(120, 150, 10))
alg_names = ["Unctrl", "RR", "MPC"]

for cap in capacities:
    for alg_name in alg_names:
        config = (alg_name, cap)
        print(config)
        if config not in sims:
            try:
                sims[config] = run_experiment(*config)
            except Exception as e:
                print(f"InfeasibilityException: {e}")
                sims[config] = None
                continue

('Unctrl', 120)
96.3855421686747
173.91304347826087
Running...
Run time: 0.5233421325683594
('RR', 120)
96.3855421686747
173.91304347826087
Running...
Run time: 2.370570659637451
('MPC', 120)
96.3855421686747
173.91304347826087
Running...
Run time: 18.35739517211914
('Unctrl', 130)
104.41767068273093
188.40579710144928
Running...
Run time: 0.4999072551727295
('RR', 130)
104.41767068273093
188.40579710144928
Running...
Run time: 2.3689653873443604
('MPC', 130)
104.41767068273093
188.40579710144928
Running...
Run time: 18.11402153968811
('Unctrl', 140)
112.44979919678714
202.89855072463766
Running...
Run time: 0.48309850692749023
('RR', 140)
112.44979919678714
202.89855072463766
Running...
Run time: 2.244719982147217
('MPC', 140)
112.44979919678714
202.89855072463766
Running...
Run time: 18.057446002960205


In [6]:
# result_dir = "results/sims/new_simulation"
# if not os.path.exists(result_dir):
#     os.makedirs(result_dir)
    
# for config, sim in sims.items():
#     name = "results/sims/new_simulation/{0}-{1}.json.gz".format(*config)
#     if not os.path.exists(name):
#         data = sim.to_json()
#         with gzip.GzipFile(name, 'w') as fout:
#             fout.write(json.dumps(data).encode('utf-8'))

#### Result Analysis

In [35]:
def calc_metrics(config, sim):
    metrics = {
        "Network Type": "three_phase",
        "Algorithm": config[0],
        "Capacity (kW)": config[1],
        "Energy Delivered (%)": analysis.proportion_of_energy_delivered(sim) * 100,
        "Demand met": analysis.proportion_of_demands_met(sim) * 100,
        "Max Utilization (%)": np.max(analysis.aggregate_power(sim)) / config[1] * 100,
        "energy_cost": analysis.energy_cost(sim),
        "total_energy_delivered": analysis.total_energy_delivered(sim),
        "Energy Delivered to priority evs (%)": analysis.proportion_of_priority_evs_energy_delivered(sim, ["session_14",
                                                                                                           "session_10",
                                                                                                           "session_13",
                                                                                                           "session_12",
                                                                                                           "session_40",
                                                                                                           "session_11",
                                                                                                           "session_29",
                                                                                                           "session_37",
                                                                                                           "session_41",
                                                                                                           "session_4",
                                                                                                           "session_5",
                                                                                                           "session_9",
                                                                                                           "session_35"
                                                                                                           ]),
        "Peak (kW)": np.max(analysis.aggregate_power(sim))
    }
    metrics["Current Unbalance"] = np.nanmean(analysis.current_unbalance(sim, ['Secondary {0}'.format(p) for p in 'ABC'], 'NEMA'))

    return metrics

In [13]:
# filter the sims dict to only include the ones that are not None
sims = {k: v for k, v in sims.items() if v is not None}

In [14]:
metrics = pd.DataFrame(calc_metrics(config, sim) for config, sim in sims.items()).round(3)
metrics.rename(columns={"Capacity (kW)": "capacity"}, inplace=True)

In [15]:
metrics

,Network Type,Algorithm,capacity,Energy Delivered (%),Demand met,Max Utilization (%),energy_cost,total_energy_delivered,Energy Delivered to priority evs (%),Peak (kW),Current Unbalance
0,three_phase,Unctrl,120,100.000,100.000,137.500,51.005,577.496,100.000,165.000,0.328
1,three_phase,RR,120,67.331,71.739,82.500,57.482,388.835,99.524,99.000,0.406
2,three_phase,MPC,120,75.307,47.826,90.450,81.037,434.896,100.000,108.540,0.309
3,three_phase,Unctrl,130,100.000,100.000,126.923,51.005,577.496,100.000,165.000,0.317
4,three_phase,RR,130,67.382,71.739,76.154,57.494,389.131,99.646,99.000,0.385
5,three_phase,MPC,130,75.379,47.826,87.004,81.204,435.311,100.000,113.105,0.329
6,three_phase,Unctrl,140,100.000,100.000,117.857,51.005,577.496,100.000,165.000,0.317
7,three_phase,RR,140,67.382,71.739,70.714,57.494,389.131,99.646,99.000,0.413
8,three_phase,MPC,140,75.367,50.000,83.607,81.199,435.242,100.000,117.050,0.364


#### Simulation with different algorithms

In [36]:
sch = {}
sch['Unctrl'] = algorithms.UncontrolledCharging()
sch['LLF'] = algorithms.SortedSchedulingAlgo(algorithms.least_laxity_first)
sch['RR'] = algorithms.RoundRobin(algorithms.first_come_first_served, continuous_inc=1)

cost_min_obj = [
                modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty),
                modified_adacharge.ObjectiveComponent(modified_adacharge.tou_energy_cost_with_pv),
                # modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty_for_priority_ev, 2),
                # modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty_without_priority_ev),
                modified_adacharge.ObjectiveComponent(modified_adacharge.quick_charge, 1e-6)
               ]

sch['MPC'] = modified_adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj, solver="MOSEK", quantize=True, reallocate=False, peak_limit=1000, max_recompute=1)

In [37]:
sims = dict()

In [38]:
warnings.simplefilter("ignore")

capacities = list(range(120, 150, 10))
alg_names = ["Unctrl", "RR", "LLF", "MPC"]

for cap in capacities:
    for alg_name in alg_names:
        config = (alg_name, cap)
        print(config)
        try:
            sims[config] = run_experiment(*config)
        except Exception as e:
            print(f"InfeasibilityException: {e}")
            sims[config] = None
            continue

('Unctrl', 120)
96.3855421686747
173.91304347826087
Running...
Run time: 1.0097718238830566
('RR', 120)
96.3855421686747
173.91304347826087
Running...
Run time: 4.28911280632019
('LLF', 120)
96.3855421686747
173.91304347826087
Running...
Run time: 2.2079646587371826
('MPC', 120)
96.3855421686747
173.91304347826087
Running...
Run time: 35.14851117134094
('Unctrl', 130)
104.41767068273093
188.40579710144928
Running...
Run time: 0.8713724613189697
('RR', 130)
104.41767068273093
188.40579710144928
Running...
Run time: 3.765174627304077
('LLF', 130)
104.41767068273093
188.40579710144928
Running...
Run time: 1.788872480392456
('MPC', 130)
104.41767068273093
188.40579710144928
Running...
Run time: 31.06535816192627
('Unctrl', 140)
112.44979919678714
202.89855072463766
Running...
Run time: 0.8798434734344482
('RR', 140)
112.44979919678714
202.89855072463766
Running...
Run time: 3.9620542526245117
('LLF', 140)
112.44979919678714
202.89855072463766
Running...
Run time: 1.8791751861572266
('MPC',

In [39]:
# filter the sims dict to only include the ones that are not None
sims = {k: v for k, v in sims.items() if v is not None}

In [40]:
metrics = pd.DataFrame(calc_metrics(config, sim) for config, sim in sims.items()).round(3)
metrics.rename(columns={"Capacity (kW)": "capacity"}, inplace=True)

In [41]:
metrics

,Network Type,Algorithm,capacity,Energy Delivered (%),Demand met,Max Utilization (%),energy_cost,total_energy_delivered,Energy Delivered to priority evs (%),Peak (kW),Current Unbalance
0,three_phase,Unctrl,120,100.000,100.000,137.500,30.311,539.054,100.000,165.000,0.306
1,three_phase,RR,120,73.416,80.435,82.500,22.253,395.754,81.497,99.000,0.421
2,three_phase,LLF,120,73.419,80.435,78.346,22.254,395.770,81.497,94.015,0.405
3,three_phase,MPC,120,79.166,58.696,81.454,23.996,426.745,100.000,97.745,0.369
4,three_phase,Unctrl,130,100.000,100.000,126.923,30.311,539.054,100.000,165.000,0.341
5,three_phase,RR,130,73.416,80.435,76.154,22.253,395.754,81.497,99.000,0.404
6,three_phase,LLF,130,73.413,80.435,74.077,22.252,395.736,81.485,96.300,0.399
7,three_phase,MPC,130,79.147,58.696,84.615,23.990,426.644,100.000,110.000,0.363
8,three_phase,Unctrl,140,100.000,100.000,117.857,30.311,539.054,100.000,165.000,0.301
9,three_phase,RR,140,73.416,80.435,70.714,22.253,395.754,81.497,99.000,0.431


In [55]:
for config, sim in sims.items():
    print(config, sim)

('Unctrl', 120) modified_simulator.modified_simulator.Simulator(network=<acnportal.contrib.acnsim.network.stochastic_network.StochasticNetwork object at 0x000001511E02B040>, scheduler=<acnportal.algorithms.uncontrolled_charging.UncontrolledCharging object at 0x000001511E01F9D0>, max_recompute=1, event_queue=<acnportal.acnsim.events.event_queue.EventQueue object at 0x000001511E01F250>, start=<datetime.datetime object at 0x000001517F89B540>, period=5, signals=<dict object at 0x000001517F8E29C0>, verbose=False, pilot_signals=<numpy.ndarray object at 0x000001515D2E8B70>, charging_rates=<numpy.ndarray object at 0x000001517FB43570>, peak=<numpy.float64 object at 0x000001511CA67430>, ev_history=<dict object at 0x000001517F8E2D00>, event_history=<list object at 0x000001511E04E2C0>, schedule_history=None, _iteration=900, _resolve=False, _last_schedule_update=899)
('RR', 120) modified_simulator.modified_simulator.Simulator(network=<acnportal.contrib.acnsim.network.stochastic_network.StochasticNe

In [56]:
plt.rcParams.update({
    'font.size': 8,
    'font.family': 'serif',
    'font.serif': ['Times New Roman'],
    'axes.linewidth': 0.8,
    'xtick.major.width': 0.8,
    'ytick.major.width': 0.8,
    'xtick.minor.width': 0.5,
    'ytick.minor.width': 0.5,
    'lines.linewidth': 1.2,
    'patch.linewidth': 0.8,
    'grid.linewidth': 0.5,
    'legend.frameon': True,
    'legend.fancybox': False,
    'legend.edgecolor': 'black',
    'legend.framealpha': 1.0
})

In [57]:
def plot_ev_charging(sim, ev, ax, label, label_auto_place=False):
    evse_index = sim.network.station_ids.index(ev.station_id)
    session_len = ev.departure - ev.arrival
    x = [sim.start + timedelta(minutes=5 * ev.arrival) + timedelta(minutes=5*i) for i in range(session_len)]
    
    # Define liner colors for different algorithms
    line_colors = {
        "Unctrl": "#1f77b4",  # blue
        "RR": "#8b12be",      # purple
        "LLF": "#ff7f0e",     # orange
        "MPC": "#2ca02c"      # green
    }

    # label names
    label_names = {
        "Unctrl": "Uncontrolled",
        "RR": "RR",
        "LLF": "LLF",
        "MPC": "MPC_AQ",
    }


    # Use Seaborn to plot the charging rates
    sns.lineplot(x=x, y=sim.charging_rates[evse_index][ev.arrival:ev.departure],
                 drawstyle='steps-post', ax=ax, label=label_names[label], color=line_colors[label], linewidth=1.2)

    # Set grid
    ax.grid(True, alpha=0.3, linewidth=0.5, linestyle='-')
    
    ax.tick_params(axis='both', which='major', labelsize=6, direction='in', 
               top=False, right=False, length=4, width=0.8)
    ax.tick_params(axis='both', which='minor', labelsize=6, direction='in',
                top=False, right=False, length=2, width=0.5) 
    
    if label == "MPC":
        ax.set_xlabel("Time of Day", fontsize=7)

    if label_auto_place:
        ax.legend(bbox_to_anchor=(0.5, 0.95), loc='center', fontsize=6)
    else:
        ax.text(0.02, 0.9, label, horizontalalignment='left', verticalalignment='top', transform=ax.transAxes, fontsize=7)

def plot_profiles(sims, cap, ev, end=None, label_auto_place=False):
    fig, axes = plt.subplots(4, 1, sharey=True, sharex=True, figsize=(3.5, 4.5), dpi=300)

    x_min = sim.start + timedelta(minutes=5 * ev.arrival)
    x_max = end if end is not None else sim.start + timedelta(minutes=5 * ev.departure)
    axes[0].set_xlim(x_min, x_max)
    axes[-1].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    fig.autofmt_xdate()

    axes[0].set_ylim(0, 35)
    axes[0].set_yticks([0, 15, 35], fontsize=6)
    
    # plt.subplots_adjust(hspace=.2)
    # Set seaborn style
    sns.set_style("whitegrid", {
        'axes.grid': True,
        'axes.linewidth': 0.8,
        'grid.linewidth': 0.5,
        'grid.alpha': 0.3
    })
    
    # Adjust x-position based on figure width (3.5 inches)
    # Use a smaller x value for narrower figures
    x_pos = 0.02 if fig.get_figwidth() > 4 else 0.01
    fig.text(x_pos, 0.55, 'Charging Current (A)', va='center', rotation='vertical', fontsize=7)
    # plt.tight_layout()
    print(ev.session_id)
    for i, alg_name in enumerate(["Unctrl", "RR", "LLF", "MPC"]):
        # label = alg_name if alg_name != "Offline" else "Offline Optimal"
        label = alg_name
        plot_ev_charging(sims[alg_name, cap], ev, axes[i], label=label, label_auto_place=label_auto_place)
        axes[i].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
        axes[i].spines['right'].set_visible(True)
        axes[i].spines['top'].set_visible(True)
    return fig

In [ ]:
evs = list(sims["RR", cap].ev_history.values())
print(evs)

In [ ]:
ev = random.choice(list(sims["RR", cap].ev_history.values()))
cap = 150
plot_profiles(sims, cap, ev, label_auto_place=True)

# Saving the plot
plt.tight_layout()
file_basename = "C:\\Users\\s3955218\\OneDrive - RMIT University\\PhD Writing\\Journal\\Simualtion_results\\ev_charging_profiles"
plt.savefig(f"{file_basename}.png", dpi=300, bbox_inches='tight')
print(f"Plot saved as {file_basename}.png")
plt.show()